<a href="https://colab.research.google.com/github/debanjanm/statistics/blob/main/01-survival%20analysis/pysurvival.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pysurvival

In [4]:
####  Importing packages
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from pysurvival.models.simulations import SimulationModel
from pysurvival.models.semi_parametric import CoxPHModel
from pysurvival.utils.metrics import concordance_index
from pysurvival.utils.display import integrated_brier_score
#%pylab inline

**DATA PRE-PROCESSING**

In [ ]:
####  loading dataset
from google.colab import files
uploaded = files.upload()

In [7]:
import io
data = io.BytesIO(uploaded['SURVIVAL.csv'])

In [8]:
dataset = pd.read_csv(data) 

In [ ]:
# Showing a few data-points 
dataset.head(5)

In [9]:
df1 = dataset.iloc[:,[2,3,5,6,7,8,9,10,11,12]].copy()
df2 = dataset.iloc[:,[2,3,4,6,7,8,9,10,11,12]].copy()

In [ ]:
df1.head(5)

In [ ]:
# Data types
df1.dtypes

In [10]:
# Creating one-hot vectors
categories = ['Metagene', 'TUMOR_STAGE', 'MENOPAUSAL_STATE',
              'TUMOR_SIZE', 'HISTOLOGICAL_SUBTYPE', 'BREAST_SURGERY']
df11 = pd.get_dummies(df1, columns=categories, drop_first=True)

In [11]:
#### Creating the modeling dataset
r1, c1 = df1.shape

In [12]:
# Creating the time and event columns
time_column = 'OS_MONTHS'
event_column = 'OS_STATUS'

# Extracting the features
features = np.setdiff1d(df11.columns, [time_column, event_column] ).tolist()

In [13]:
# Building training and testing sets #
index_train, index_test = train_test_split( range(r1), test_size = 0.2)
data_train = df11.loc[index_train].reset_index( drop = True )
data_test  = df11.loc[index_test].reset_index( drop = True )

# Creating the X, T and E inputs
X_train, X_test = data_train[features], data_test[features]
T_train, T_test = data_train[time_column], data_test[time_column]
E_train, E_test = data_train[event_column], data_test[event_column]

**Cox Proportional Hazard model**
*Standard CoxPH*

In [ ]:
#### Creating an instance of the Cox PH model and fitting the data.
# Building the model
coxph = CoxPHModel()
coxph.fit(X_train, T_train, E_train, lr=0.5, l2_reg=1e-2, init_method='zeros')

In [ ]:
#### Cross Validation / Model Performances
c_index = concordance_index(coxph, X_test, T_test, E_test) #0.55
print('C-index: {:.2f}'.format(c_index))

ibs = integrated_brier_score(coxph, X_test, T_test, E_test, t_max=10,
            figure_size=(20, 6.5) )
print('IBS: {:.2f}'.format(ibs))

**Cox Proportional Hazard model**
*DeepSurv/Nonlinear CoxPH*

In [17]:
from pysurvival.models.semi_parametric import NonLinearCoxPHModel

In [ ]:
#### Creating an instance of the NonLinear CoxPH model and fitting the data.

# Defining the MLP structure. Here we will build a 1-hidden layer 
# with 150 units and `BentIdentity` as its activation function
structure = [ {'activation': 'BentIdentity', 'num_units': 150},  ]

# Building the model
nonlinear_coxph = NonLinearCoxPHModel(structure=structure)
nonlinear_coxph.fit(X_train, T_train, E_train, lr=1e-3, init_method='xav_uniform')


#### 5 - Cross Validation / Model Performances
c_index = concordance_index(nonlinear_coxph, X_test, T_test, E_test) #0.56
print('C-index: {:.2f}'.format(c_index))

ibs = integrated_brier_score(nonlinear_coxph, X_test, T_test, E_test, t_max=10,
            figure_size=(20, 6.5) )
print('IBS: {:.2f}'.format(ibs))

**Multi-Task Logistic Regression (MTLR)**
*Linear MTLR model*

In [19]:
from pysurvival.models.multi_task import LinearMultiTaskModel

In [ ]:
#### Creating an instance of the Linear MTLR model and fitting the data.
# Building the model
l_mtlr = LinearMultiTaskModel(bins=50)
l_mtlr.fit(X_train, T_train, E_train, lr=1e-3, init_method='orthogonal')


#### 5 - Cross Validation / Model Performances
c_index = concordance_index(l_mtlr, X_test, T_test, E_test) #0.52
print('C-index: {:.2f}'.format(c_index))

ibs = integrated_brier_score(l_mtlr, X_test, T_test, E_test, t_max=30,
            figure_size=(20, 6.5) )
print('IBS: {:.2f}'.format(ibs))

**Multi-Task Logistic Regression (MTLR)**
*Neural MTLR model*

In [21]:
from pysurvival.models.multi_task import NeuralMultiTaskModel

In [ ]:
#### Creating an instance of the Neural MTLR model and fitting the data.

# Defining the MLP structure. Here we will build a 1-hidden layer 
# with 150 units and `Swish` as its activation function
structure = [ {'activation': 'ReLU', 'num_units': 150},  ]

# Building the model
n_mtlr = NeuralMultiTaskModel(structure=structure, bins=150)
n_mtlr.fit(X_train, T_train, E_train, lr=1e-3, num_epochs = 500,
           init_method='orthogonal', optimizer = 'rprop')


#### 5 - Cross Validation / Model Performances
c_index = concordance_index(n_mtlr, X_test, T_test, E_test) #0.51
print('C-index: {:.2f}'.format(c_index))

ibs = integrated_brier_score(n_mtlr, X_test, T_test, E_test, t_max=30,
            figure_size=(20, 6.5) )
print('IBS: {:.2f}'.format(ibs))

**Non-Parametric**
*Kaplan Meier model*

In [23]:
from pysurvival.models.non_parametric import KaplanMeierModel
from pysurvival.utils.display import display_non_parametric

In [ ]:
T = df11.iloc[:,1]
E = df11.iloc[:,0]

# Initializing the KaplanMeierModel
km_model = KaplanMeierModel()

# Fitting the model 
km_model.fit(T, E, alpha=0.95)

# Displaying the survival function and confidence intervals
display_non_parametric(km_model)

**Non-Parametric**
*Smooth Kaplan Meier model*

In [26]:
from pysurvival.models.non_parametric import SmoothKaplanMeierModel

In [ ]:
# Initializing the SmoothKaplanMeierModel
skm_model = SmoothKaplanMeierModel(bandwidth=1., kernel='Cosine')

# Fitting the model and display the survival function and confidence intervals
skm_model.fit(T, E, alpha=0.95)

# Displaying the survival function and confidence intervals
display_non_parametric(skm_model)

**Parametric models**

In [29]:
from pysurvival.models.parametric import GompertzModel
from pysurvival.models.parametric import ExponentialModel
from pysurvival.models.parametric import WeibullModel
from pysurvival.models.parametric import LogLogisticModel
from pysurvival.models.parametric import LogNormalModel

*Gompertz*

In [ ]:
#### 4 - Creating an instance of the Gompertz model and fitting the data.
# Building the model
gomp_model = GompertzModel()
gomp_model.fit(X_train, T_train, E_train, lr=1e-2, init_method='zeros',
    optimizer ='adam', l2_reg = 1e-3, num_epochs=2000)


#### 5 - Cross Validation / Model Performances
c_index = concordance_index(gomp_model, X_test, T_test, E_test) #0.55
print('C-index: {:.2f}'.format(c_index))

ibs = integrated_brier_score(gomp_model, X_test, T_test, E_test, t_max=30,
            figure_size=(20, 6.5) )
print('IBS: {:.2f}'.format(ibs))

*Exponential*

In [ ]:
#### 4 - Creating an instance of the Gompertz model and fitting the data.
# Building the model
exp_model = ExponentialModel()
exp_model.fit(X_train, T_train, E_train, lr=1e-2, init_method='zeros',
    optimizer ='adam', l2_reg = 1e-3, num_epochs=2000)


#### 5 - Cross Validation / Model Performances
c_index = concordance_index(exp_model, X_test, T_test, E_test) #0.56
print('C-index: {:.2f}'.format(c_index))

ibs = integrated_brier_score(exp_model, X_test, T_test, E_test, t_max=30,
            figure_size=(20, 6.5) )
print('IBS: {:.2f}'.format(ibs))

*Log-Logistic*

In [ ]:
#### 4 - Creating an instance of the Gompertz model and fitting the data.
# Building the model
loglog_model = LogLogisticModel()
loglog_model.fit(X_train, T_train, E_train, lr=1e-2, init_method='zeros',
    optimizer ='adam', l2_reg = 1e-3, num_epochs=2000)


#### 5 - Cross Validation / Model Performances
c_index = concordance_index(loglog_model, X_test, T_test, E_test) #0.56
print('C-index: {:.2f}'.format(c_index))

ibs = integrated_brier_score(loglog_model, X_test, T_test, E_test, t_max=30,
            figure_size=(20, 6.5) )
print('IBS: {:.2f}'.format(ibs))